In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datetime as dt
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
import pandas as pd

[2022-08-15 11:32:51,116] {utils.py:159} INFO - NumExpr defaulting to 4 threads.


In [3]:
def CSVToJson():
    df=pd.read_CSV('data.csv')
    for i,r in df.iterrows():
        print(r['name'])
        df.to_JSON('fromAirflow.JSON',orient='records')

In [4]:
default_args = {
 'owner': 'tolani',
 'start_date': dt.datetime(2022, 8, 15),
 'retries': 1,
 'retry_delay': dt.timedelta(minutes=5),
}

In [5]:
with DAG('MyCSVDAG',
 default_args=default_args,
 schedule_interval=timedelta(minutes=5),
 # '0 * * * *',
 ) as dag:
    print_starting = BashOperator(task_id='starting',bash_command='echo "I am reading the CSV now....."')
    CSVJson = PythonOperator(task_id='convertCSVtoJson',python_callable=CSVToJson)

In [6]:
print_starting .set_downstream(CSVJson)

In [7]:
CSVJson.set_upstream(print_starting)

In [8]:
print_starting >> CSVJson

<Task(PythonOperator): convertCSVtoJson>